In [2]:
#RAN
from google.colab import files
import zipfile
import os
from pathlib import Path
import shutil

# Upload ZIP files
uploaded = files.upload()  # Expect images.zip and labels.zip

# Define paths
base_path = Path("brain_tumor_data")
images_path = base_path / "images"
labels_path = base_path / "labels"

# Create folders
images_path.mkdir(parents=True, exist_ok=True)
labels_path.mkdir(parents=True, exist_ok=True)

# Function to unzip and flatten
def unzip_and_flatten(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

    # Move all files to root of extract_to
    for root, _, files in os.walk(extract_to):
        for file in files:
            src = os.path.join(root, file)
            dst = os.path.join(extract_to, file)
            if src != dst:
                shutil.move(src, dst)

    # Remove subdirectories
    for item in os.listdir(extract_to):
        item_path = os.path.join(extract_to, item)
        if os.path.isdir(item_path):
            shutil.rmtree(item_path)

# Unzip uploaded files
unzip_and_flatten("images.zip", images_path)
unzip_and_flatten("labels.zip", labels_path)

In [3]:
#RAN
import pandas as pd

# List image and label files
image_files = list(images_path.glob("*"))
label_files = list(labels_path.glob("*"))

# Match by base filename (without extension)
def get_base_filename(path):
    return path.stem

image_map = {get_base_filename(p): p for p in image_files}
label_map = {get_base_filename(p): p for p in label_files}

matched_data = []
for base_name, label_path in label_map.items():
    if base_name in image_map:
        with open(label_path, 'r') as f:
            content = f.read().strip()
            if content:
                first_class = int(content.split('\n')[0].split()[0])
                matched_data.append({
                    "filename": base_name,
                    "image_path": str(image_map[base_name]),
                    "label_path": str(label_path),
                    "has_tumor": first_class == 1
                })

# Convert to DataFrame
matched_df = pd.DataFrame(matched_data)

# Count tumor vs no tumor
tumor_count = matched_df["has_tumor"].sum()
no_tumor_count = len(matched_df) - tumor_count

print(f"Tumor Images: {tumor_count}")
print(f"No Tumor Images: {no_tumor_count}")
print(f"Total Matched: {len(matched_df)}")

# Optional: preview
matched_df.head()

Tumor Images: 459
No Tumor Images: 419
Total Matched: 878


,filename,image_path,label_path,has_tumor
0,00313_126,brain_tumor_data/images/00313_126.jpg,brain_tumor_data/labels/00313_126.txt,True
1,97 (2),brain_tumor_data/images/97 (2).jpg,brain_tumor_data/labels/97 (2).txt,True
2,00090_112,brain_tumor_data/images/00090_112.jpg,brain_tumor_data/labels/00090_112.txt,False
3,00172_108,brain_tumor_data/images/00172_108.jpg,brain_tumor_data/labels/00172_108.txt,False
4,00148_19,brain_tumor_data/images/00148_19.jpg,brain_tumor_data/labels/00148_19.txt,False


In [4]:
#RAN
from sklearn.model_selection import train_test_split
import pandas as pd

# Assume matched_df is already created
# Stratified split to maintain tumor/no-tumor ratio
train_df, test_df = train_test_split(
    matched_df,
    test_size=0.2,
    stratify=matched_df["has_tumor"],
    random_state=42
)

# Reset index for convenience
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# Print summary
print(f"Training Set: {len(train_df)} images")
print(f"  - Tumor: {train_df['has_tumor'].sum()}")
print(f"Test Set: {len(test_df)} images")
print(f"  - Tumor: {test_df['has_tumor'].sum()}")

Training Set: 702 images
  - Tumor: 367
Test Set: 176 images
  - Tumor: 92


In [6]:
!pip install torch torchvision albumentations opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [7]:
# Then re-install clean versions:
!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 25.9 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0


In [ ]:
# Step 1: Downgrade NumPy
!pip install numpy==1.24.4 --force-reinstall

# Step 2: Restart your runtime after this command runs
import os
os.kill(os.getpid(), 9)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 97.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
blosc2 3.2.1 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.


In [6]:
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import random

# -------------------- Seed Everything --------------------
np.random.seed(42)
torch.manual_seed(42)
random.seed(42)

# -------------------- Dataset Class --------------------
class TumorClassificationDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = cv2.imread(row['image_path'])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = int(row['has_tumor'])

        if self.transform:
            image = self.transform(image=image)['image']

        return image, label

# -------------------- Transforms --------------------
train_transform = A.Compose([
    A.Resize(128, 128),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=15, p=0.3),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.Normalize(),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(128, 128),
    A.Normalize(),
    ToTensorV2()
])

# -------------------- Model --------------------
class TumorClassifier(nn.Module):
    def __init__(self):
        super(TumorClassifier, self).__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 16 * 16, 128), nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 1), nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)

# -------------------- Load Data --------------------
# Use your real train_df here. Should include columns: image_path and has_tumor.
# train_df = pd.read_csv(...)  # Example

# Split validation from train_df
val_frac = 0.2
val_count = int(len(train_df) * val_frac)
train_df_split = train_df.iloc[:-val_count].reset_index(drop=True)
val_df_split = train_df.iloc[-val_count:].reset_index(drop=True)

train_dataset = TumorClassificationDataset(train_df_split, transform=train_transform)
val_dataset = TumorClassificationDataset(val_df_split, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# -------------------- Handle Class Imbalance --------------------
all_labels = train_df_split['has_tumor'].tolist()
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(all_labels), y=all_labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

# -------------------- Training Setup --------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TumorClassifier().to(device)
criterion = nn.BCELoss(weight=class_weights_tensor[1].to(device))
optimizer = optim.Adam(model.parameters(), lr=0.001)

# -------------------- Early Stopping Setup --------------------
best_val_loss = float('inf')
patience, counter = 5, 0

# -------------------- Training Loop --------------------
for epoch in range(100):
    model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.float().unsqueeze(1).to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()


    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).squeeze()
            loss = criterion(outputs.unsqueeze(1), labels.float().unsqueeze(1))
            val_loss += loss.item()

            preds = (outputs > 0.5).int()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_accuracy = correct / total if total > 0 else 0
    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f} | Val Loss={val_loss:.4f} | Val Accuracy: {val_accuracy:.4f}")

    # Checkpoint
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), "tumor_cnn_best.pth")
        print("Saved new best model.")
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered.")
            break
# -------------------- Final Evaluation --------------------
print("\nLoading best saved model for final validation accuracy...")
model.load_state_dict(torch.load("tumor_cnn_best.pth"))
model.eval()

correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).squeeze()
        preds = (outputs > 0.5).int()

        correct += (preds == labels).sum().item()
        total += labels.size(0)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

final_accuracy = correct / total if total > 0 else 0
print(f"\n Final Validation Accuracy: {final_accuracy:.4f} on {total} samples")

# Optional: Print full classification report
from sklearn.metrics import classification_report
print("\n Final Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:87: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


Epoch 1: Train Loss=24.0736 | Val Loss=5.9849 | ✅ Val Accuracy: 0.5571
Saved new best model.
Epoch 2: Train Loss=23.6044 | Val Loss=6.0447 | ✅ Val Accuracy: 0.5429
Epoch 3: Train Loss=23.6868 | Val Loss=6.0630 | ✅ Val Accuracy: 0.4857
Epoch 4: Train Loss=23.8573 | Val Loss=6.0096 | ✅ Val Accuracy: 0.5357
Epoch 5: Train Loss=22.9891 | Val Loss=6.2731 | ✅ Val Accuracy: 0.5357
Epoch 6: Train Loss=23.4344 | Val Loss=5.8911 | ✅ Val Accuracy: 0.5643
Saved new best model.
Epoch 7: Train Loss=22.8998 | Val Loss=5.9540 | ✅ Val Accuracy: 0.5429
Epoch 8: Train Loss=23.1689 | Val Loss=5.9531 | ✅ Val Accuracy: 0.5429
Epoch 9: Train Loss=22.4077 | Val Loss=5.5336 | ✅ Val Accuracy: 0.5643
Saved new best model.
Epoch 10: Train Loss=21.6666 | Val Loss=5.3858 | ✅ Val Accuracy: 0.6500
Saved new best model.
Epoch 11: Train Loss=21.4585 | Val Loss=5.4330 | ✅ Val Accuracy: 0.6357
Epoch 12: Train Loss=21.5530 | Val Loss=5.5048 | ✅ Val Accuracy: 0.5929
Epoch 13: Train Loss=20.5366 | Val Loss=5.3801 | ✅ Val Ac

In [ ]:
import shutil


shutil.rmtree('brain_tumor_yolo_runs')
shutil.rmtree('yolo_data')

In [7]:
# STEP 0: Setup UTF-8 Locale
import os
os.environ['LANG'] = 'en_US.UTF-8'

# STEP 1: Install Ultralytics
!pip install -q ultralytics

# STEP 2: Import Required Libraries
from ultralytics import YOLO
from pathlib import Path
import pandas as pd
import shutil
from google.colab import files

# STEP 3: Prepare YOLO Directory Structure
yolo_base = Path("yolo_data")
for split in ["train", "val"]:
    (yolo_base / split / "images").mkdir(parents=True, exist_ok=True)
    (yolo_base / split / "labels").mkdir(parents=True, exist_ok=True)

# STEP 4: Save images and labels (only for class 1 → 0 for YOLO)
def save_yolo_format(df, split):
    for _, row in df.iterrows():
        image_path = Path(row["image_path"])
        label_path = Path(row["label_path"])
        if not label_path.exists():
            continue

        with open(label_path, "r") as f:
            lines = f.read().strip().splitlines()

        # Convert class 1 → 0 (YOLO expects classes starting from 0)
        tumor_lines = []
        for line in lines:
            parts = line.strip().split()
            if parts and parts[0] == "1":
                parts[0] = "0"
                tumor_lines.append(" ".join(parts))

        if not tumor_lines:
            continue

        shutil.copy(image_path, yolo_base / split / "images" / image_path.name)
        with open(yolo_base / split / "labels" / f"{image_path.stem}.txt", "w") as f:
            f.write("\n".join(tumor_lines))

# Step 5: Apply to train and test splits
save_yolo_format(train_df, "train")
save_yolo_format(test_df, "val")

# Step 6: Create YAML file
yaml_path = yolo_base / "tumor.yaml"
yaml_content = f"""
path: {yolo_base.resolve()}
train: train/images
val: val/images
names:
  0: tumor
"""
with open(yaml_path, "w") as f:
    f.write(yaml_content.strip())

# Step 7: Train YOLOv8 with Enhancements
model = YOLO("yolov8s.pt")  # Better than yolov8n.pt

model.train(
    data=str(yaml_path),
    epochs=100,
    imgsz=512,
    batch=16,
    lr0=0.002,
    momentum=0.937,
    weight_decay=0.0005,
    dropout=0.2,
    optimizer='AdamW',
    patience=20,
    cos_lr=True,
    mixup=0.1,
    auto_augment='randaugment',
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    translate=0.1,
    scale=0.4,
    shear=2.0,
    perspective=0.0005,
    flipud=0.1,
    fliplr=0.5,
    mosaic=1.0,
    label_smoothing=0.01,
    warmup_epochs=5,
    box=0.05,
    cls=0.3,
    dfl=1.5,
    project='brain_tumor_yolo_runs',
    name='improved_detector',
    seed=42,
    device='cuda',
    verbose=True
)

# Step 8: Load Best Model
best_model_path = "brain_tumor_yolo_runs/improved_detector/weights/best.pt"
model = YOLO(best_model_path)

# Step 9: Evaluate
metrics = model.val()
precision = metrics.box.mp
recall = metrics.box.mr
f1_score = 2 * (precision * recall) / (precision + recall + 1e-6)

# Step 10: Save Evaluation Metrics
results = {
    "Precision": round(precision, 4),
    "Recall": round(recall, 4),
    "F1 Score": round(f1_score, 4),
    "mAP@0.5": round(metrics.box.map50, 4),
    "mAP@0.5:0.95": round(metrics.box.map, 4)
}

metrics_df = pd.DataFrame([results])
metrics_df.to_csv("tumor_yolo_metrics.csv", index=False)
print(metrics_df)

# Step 11: Predict for Manual Inspection
model.predict(source="yolo_data/val/images", save=True, conf=0.25)
print(" Predictions saved to: runs/detect/predict")

#  Step 12: Download Model & Metrics
files.download("tumor_yolo_metrics.csv")
files.download(best_model_path)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 27.7 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 21.5M/21.5M [00:00<00:00, 214MB/s]


WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.105 🚀 Python-3.11.11 torch-2.1.0+cu118 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=yolo_data/tumor.yaml, epochs=100, time=None, patience=20, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=brain_tumor_yolo_runs, name=improved_detector, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_t

100%|██████████| 755k/755k [00:00<00:00, 19.8MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

...

                   all         92        100      0.948      0.918      0.962      0.692
Speed: 1.6ms preprocess, 7.0ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/val
   Precision  Recall  F1 Score  mAP@0.5  mAP@0.5:0.95
0     0.9483  0.9178    0.9328   0.9616        0.6923

image 1/92 /content/yolo_data/val/images/00054_145.jpg: 512x512 1 tumor, 9.6ms
image 2/92 /content/yolo_data/val/images/00056_239.jpg: 512x512 1 tumor, 11.1ms
image 3/92 /content/yolo_data/val/images/00058_141.jpg: 512x512 1 tumor, 9.7ms
image 4/92 /content/yolo_data/val/images/00059_122.jpg: 512x512 1 tumor, 9.6ms
image 5/92 /content/yolo_data/val/images/00060_72.jpg: 512x512 1 tumor, 9.6ms
image 6/92 /content/yolo_data/val/images/00063_217.jpg: 512x512 1 tumor, 9.8ms
image 7/92 /content/yolo_data/val/images/00066_221.jpg: 512x512 1 tumor, 9.7ms
image 8/92 /content/yolo_data/val/images/00066_278.jpg: 512x512 1 tumor, 10.6ms
image 9/92 /content/yolo_data/val/images/00071_240.jpg

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
import os
import cv2
import torch
import numpy as np
from ultralytics import YOLO
from albumentations.pytorch import ToTensorV2
import albumentations as A
from pathlib import Path
from torchvision import transforms
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cnn_model = TumorClassifier().to(device)
cnn_model.load_state_dict(torch.load("tumor_cnn_best.pth", map_location=device))
cnn_model.eval()

# ---------------- Load YOLOv8 Model ---------------- #
yolo_model = YOLO("/content/brain_tumor_yolo_runs/improved_detector/weights/best.pt")

# ---------------- Transforms ---------------- #
cnn_transform = A.Compose([
    A.Resize(128, 128),
    A.Normalize(),
    ToTensorV2()
])

# ---------------- Helper Function ---------------- #
def run_pipeline(image_path, show_result=True, save_dir="runs/pipeline_results"):
    # Prepare image
    original_img = cv2.imread(image_path)
    rgb_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    tensor_img = cnn_transform(image=rgb_img)['image'].unsqueeze(0).to(device)

    # Classify
    with torch.no_grad():
        pred = cnn_model(tensor_img).item()

    # Decide based on threshold
    print(f"\n {Path(image_path).name} - Tumor Probability: {pred:.4f}")
    if pred < 0.5:
        print(" No tumor detected.")
        if show_result:
            plt.imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
            plt.title("No Tumor Detected")
            plt.axis("off")
            plt.show()
        return

    print("Tumor detected! Running YOLOv8 for localization...")

    # Run YOLO
    results = yolo_model.predict(source=image_path, save=True, save_txt=False, conf=0.25)

    # Display/save result
    if show_result:
        result_img = results[0].plot()
        plt.imshow(result_img)
        plt.title("Tumor Detection")
        plt.axis("off")
        plt.show()

    print(f"YOLOv8 prediction saved in: {results[0].save_dir}")
test_folder = "/content/runs/kaggle"
for img_path in Path(test_folder).glob("*.jpg"):
    run_pipeline(str(img_path))
# ---------------- Run Example ---------------- #
# Example usage: replace path below with any test image
# run_pipeline("yolo_data/val/images/example.jpg")

In [ ]:
import shutil

shutil.rmtree('/content/runs/kaggle')